In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from ipywidgets import IntProgress
from IPython.display import display
import pandas as pd
pd.options.display.max_columns = 30

In [6]:
%%time

PATH = "C:/Program Files (x86)/chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get("https://www.nba.com/stats/teams/boxscores/")

matchData  = {
    'Team':[],
    'MatchUp':[],
    'Date':[],
    'W/L':[],
    'Min':[],
    'Points':[],
    'FGM':[], # Field Goals Made
    'FGA':[], # Field Goals Attempt
    'FGP':[], # Field Goals Percentage
    '3PM':[], # 3 Points Made
    '3PA':[], # 3 Points Attempt
    '3PP':[], # 3 Points Percentage
    'FTM':[], # Free Throw Made
    'FTA':[], # Free Throw Attempt
    'FTP':[], # Free Throw Percentage
    'OREB':[],# Offensive Rebounds
    'DREB':[],# Defensive Rebounds
    'REB':[], # Rebound
    'AST':[], # Assists
    'STL':[], # Steals
    'BLK':[], # Blocks
    'TOV':[], # Turnover
    'PF':[],  # Personal Fouls
    'PN':[],  # +/-
}
cols = list(matchData.keys())

total = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.stats-table-pagination__info"))).text
pages = int(total.split('of ')[-1])
total = int(total.split(' Rows')[0])

nextButton = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"a.stats-table-pagination__next")))

pbar = IntProgress(min=0, max=total)
display(pbar)

for page in range(pages):
    if page != 0:
        nextButton.click()

    rowsInPage = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"tr[data-ng-repeat='(i, row) in page track by row.$hash']")))
    rowsInPage = int(len(rowsInPage)/2)

    for i in range(rowsInPage):
        result = []
        dataRow = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,f"tr[data-ng-repeat='(i, row) in page track by row.$hash'][index='{i}'] td")))

        for ele in dataRow: 
            if ele.text != '': result.append(ele.text)
        result.pop()
        
        for j in range(len(cols)):
            matchData[cols[j]].append(result[j])
            
        pbar.value+=1

matchData = pd.DataFrame(matchData)
driver.quit()
matchData.to_csv('MatchData.csv')

IntProgress(value=0, max=2138)

Wall time: 15min 30s
